# IoT Device Lifecycle

Manage virtual IoT devices and demonstrate **EST enrollment** workflows via the IoT Client API.

## EST-First Enrollment Strategy

The IoT Client simulator uses an EST-first enrollment strategy:

1. Probes `/.well-known/est/cacerts` on the EST Sub-CA to check EST availability
2. If EST is available, enrolls via `/.well-known/est/simpleenroll` (RFC 7030)
3. Falls back to the Dogtag REST API (`/ca/rest/certrequests`) if EST is unavailable

## Device Lifecycle States

```
Created ──▶ Enrolled (has certificate) ──▶ Renewed (new certificate)
                    │
                    └─▶ Revoked (via EDA event)
```

## EST Sub-CAs per PKI Hierarchy

| PKI Type | EST CA Container | Port | EST Endpoint |
|----------|-----------------|------|-------------|
| RSA-4096 | dogtag-est-ca | 8447 | `/.well-known/est/simpleenroll` |
| ECC P-384 | dogtag-ecc-est-ca | 8466 | `/.well-known/est/simpleenroll` |
| ML-DSA-87 | dogtag-pq-est-ca | 8456 | `/.well-known/est/simpleenroll` |

## Configuration

The IoT Client API is accessible from the Jupyter container by hostname on the lab network.

| Variable | Default | Description |
|----------|---------|-------------|
| `IOT_CLIENT_URL` | `http://iot-client.cert-lab.local:8000` | IoT Client API base URL |

In [ ]:
import json
from datetime import datetime

import httpx
import pandas as pd
from IPython.display import display

IOT_CLIENT_URL = "http://iot-client.cert-lab.local:8000"

print(f"IoT Client URL: {IOT_CLIENT_URL}")

## Health & CA Availability

Check which CAs and EST endpoints are reachable per PKI type.

In [ ]:
try:
    resp = httpx.get(f"{IOT_CLIENT_URL}/health", timeout=10)
    resp.raise_for_status()
    health = resp.json()
    print("IoT Client Health:")
    print(json.dumps(health, indent=2))
except Exception as e:
    print(f"Failed to reach IoT Client: {e}")
    print("Make sure the iot-client container is running.")

## Create Device

Create a virtual IoT device. Modify `DEVICE_TYPE` and `DEVICE_PKI` below.

In [ ]:
DEVICE_TYPE = "sensor"   # sensor, actuator, gateway, camera
DEVICE_PKI = "rsa"       # rsa, ecc, pqc

try:
    resp = httpx.post(
        f"{IOT_CLIENT_URL}/devices",
        json={"device_type": DEVICE_TYPE, "pki_type": DEVICE_PKI},
        timeout=10,
    )
    resp.raise_for_status()
    device = resp.json()
    device_id = device.get("device_id") or device.get("id")
    print(f"Device created: {device_id}")
    print(json.dumps(device, indent=2))
except Exception as e:
    print(f"Failed to create device: {e}")

## Enroll Device

Enroll a device for a certificate. The IoT Client will attempt EST first, then fall back to REST API.

**Set `ENROLL_DEVICE_ID`** to the ID from the cell above (or any existing device).

In [ ]:
# Use the device_id from the creation cell, or set manually
ENROLL_DEVICE_ID = device_id if 'device_id' in dir() else "SET_ME"

try:
    resp = httpx.post(
        f"{IOT_CLIENT_URL}/devices/{ENROLL_DEVICE_ID}/enroll",
        timeout=30,
    )
    resp.raise_for_status()
    result = resp.json()
    print(f"Enrollment result for {ENROLL_DEVICE_ID}:")
    print(json.dumps(result, indent=2))
except Exception as e:
    print(f"Failed to enroll device: {e}")

## View Certificate

Retrieve and display the certificate issued to a device.

In [ ]:
VIEW_DEVICE_ID = device_id if 'device_id' in dir() else "SET_ME"

try:
    resp = httpx.get(
        f"{IOT_CLIENT_URL}/devices/{VIEW_DEVICE_ID}/certificate",
        timeout=10,
    )
    resp.raise_for_status()
    cert_data = resp.json()
    print(f"Certificate for device {VIEW_DEVICE_ID}:")
    print(json.dumps(cert_data, indent=2))
except Exception as e:
    print(f"Failed to get certificate: {e}")

## Bulk Enrollment

Create and enroll multiple devices at once using the bulk enrollment endpoint.

In [ ]:
BULK_COUNT = 5           # number of devices to create
BULK_TYPE = "sensor"     # device type
BULK_PKI = "rsa"         # rsa, ecc, pqc

try:
    resp = httpx.post(
        f"{IOT_CLIENT_URL}/bulk/enroll",
        json={"count": BULK_COUNT, "device_type": BULK_TYPE, "pki_type": BULK_PKI},
        timeout=60,
    )
    resp.raise_for_status()
    result = resp.json()

    if isinstance(result, list):
        df = pd.DataFrame(result)
        print(f"Bulk enrollment results: {len(df)} devices")
        display(df)
    elif isinstance(result, dict):
        devices = result.get("devices", result.get("results", [result]))
        if isinstance(devices, list):
            df = pd.DataFrame(devices)
            print(f"Bulk enrollment results: {len(df)} devices")
            display(df)
        else:
            print(json.dumps(result, indent=2))
except Exception as e:
    print(f"Failed bulk enrollment: {e}")

## Device Inventory

List all virtual IoT devices with their status, enrollment method (EST vs REST), and timestamps.

In [ ]:
try:
    resp = httpx.get(f"{IOT_CLIENT_URL}/devices", timeout=10)
    resp.raise_for_status()
    devices = resp.json()

    if isinstance(devices, list):
        items = devices
    elif isinstance(devices, dict):
        items = devices.get("devices", [devices])
    else:
        items = []

    if items:
        df = pd.DataFrame(items)
        print(f"Total devices: {len(df)}")
        display(df)
    else:
        print("No devices found. Create some using the cells above.")
except Exception as e:
    print(f"Failed to list devices: {e}")

## Enrollment Statistics

Aggregate statistics broken down by PKI type and enrollment status.

In [ ]:
try:
    resp = httpx.get(f"{IOT_CLIENT_URL}/statistics", timeout=10)
    resp.raise_for_status()
    stats = resp.json()
    print("Enrollment Statistics:")
    print(json.dumps(stats, indent=2))
except Exception as e:
    print(f"Failed to get statistics: {e}")

## Renew Device Certificate

Renew an existing device's certificate. The device must have been previously enrolled.

In [ ]:
RENEW_DEVICE_ID = device_id if 'device_id' in dir() else "SET_ME"

try:
    resp = httpx.post(
        f"{IOT_CLIENT_URL}/devices/{RENEW_DEVICE_ID}/renew",
        timeout=30,
    )
    resp.raise_for_status()
    result = resp.json()
    print(f"Renewal result for {RENEW_DEVICE_ID}:")
    print(json.dumps(result, indent=2))
except Exception as e:
    print(f"Failed to renew certificate: {e}")